In [1]:
import iris

filename = iris.sample_data_path('uk_hires.pp')
cubes = iris.load(filename)
print(cubes)

0: air_potential_temperature / (K)     (time: 3; model_level_number: 7; grid_latitude: 204; grid_longitude: 187)
1: surface_altitude / (m)              (grid_latitude: 204; grid_longitude: 187)


In [2]:
cube1 = cubes[0]

In [4]:
print(cube1)

air_potential_temperature / (K)     (time: 3; model_level_number: 7; grid_latitude: 204; grid_longitude: 187)
     Dimension coordinates:
          time                           x                      -                 -                    -
          model_level_number             -                      x                 -                    -
          grid_latitude                  -                      -                 x                    -
          grid_longitude                 -                      -                 -                    x
     Auxiliary coordinates:
          forecast_period                x                      -                 -                    -
          level_height                   -                      x                 -                    -
          sigma                          -                      x                 -                    -
          surface_altitude               -                      -                 x                

In [6]:
# time dimensionを固定
print(cube1[0])

air_potential_temperature / (K)     (model_level_number: 7; grid_latitude: 204; grid_longitude: 187)
     Dimension coordinates:
          model_level_number                           x                 -                    -
          grid_latitude                                -                 x                    -
          grid_longitude                               -                 -                    x
     Auxiliary coordinates:
          level_height                                 x                 -                    -
          sigma                                        x                 -                    -
          surface_altitude                             -                 x                    x
     Derived coordinates:
          altitude                                     x                 x                    x
     Scalar coordinates:
          forecast_period: 6.0 hours
          forecast_reference_time: 2009-11-19 04:00:00
          time: 2009-11-19 1

In [7]:
# time dimensionと高度を固定
print(cube1[0, 0])

air_potential_temperature / (K)     (grid_latitude: 204; grid_longitude: 187)
     Dimension coordinates:
          grid_latitude                           x                    -
          grid_longitude                          -                    x
     Auxiliary coordinates:
          surface_altitude                        x                    x
     Derived coordinates:
          altitude                                x                    x
     Scalar coordinates:
          forecast_period: 6.0 hours
          forecast_reference_time: 2009-11-19 04:00:00
          level_height: 5.0 m, bound=(0.0, 13.3333) m
          model_level_number: 1
          sigma: 0.999424, bound=(1.0, 0.998464)
          time: 2009-11-19 10:00:00
     Attributes:
          STASH: m01s00i004
          source: Data from Met Office Unified Model
          um_version: 7.3


resolutionなので、timeは荒くしない。荒くするのは3次元の空間。

でも高度は一般にそこまでポイントがないので、とりあえず荒くしなくてよくって、二次元平面を荒くすることを考える。

引数には落とす線の割合を$[0, 1]$で指定

In [8]:
drop_rate = 0.1

In [53]:
cube1.coords()[0].name()

'time'

In [15]:
lat_num = len(cube1.coord("grid_latitude").points)
lon_num = len(cube1.coord("grid_longitude").points)

In [16]:
lat_num

204

In [19]:
import math

In [35]:
span = math.ceil((lat_num - math.ceil((lat_num*drop_rate))) /(math.ceil((lat_num*drop_rate)) + 1))

9

In [36]:
last = math.ceil((lat_num - span)/(span+1))
last

20

In [38]:
last_ind = span + (last - 1)*(span + 1)
last_ind

199

In [48]:
# 抜く奴はこれ
drop_ind = np.linspace(span, last_ind, 20)

In [51]:
# 残るのはこれ
remain_ind = set(range(lat_num)).difference(set(drop_ind))

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 200,
 201,
 202,
 

In [1]:
# extractとかいうメソッドがあったのでそっちを使う
# 緯度経度の名前を"grid_latitude"と"grid_longitude"に変更してから使うこと。
import math
import numpy as np
from iris.coords import DimCoord
from iris.cube import Cube

def lat_id(cube, drop_rate):
    lat_num = len(cube.coord("grid_latitude").points)
    span = math.ceil((lat_num - math.ceil((lat_num*drop_rate))) /(math.ceil((lat_num*drop_rate)) + 1))
    last = math.ceil((lat_num - span)/(span+1))
    last_ind = span + (last - 1)*(span + 1)
    
    drop_ind = np.linspace(span, last_ind, last)
    return list(set(range(lat_num)).difference(set(drop_ind)))

def lon_id( cube, drop_rate):
    lon_num = len(cube.coord("grid_longitude").points)
    span = math.ceil((lon_num - math.ceil((lon_num*drop_rate))) /(math.ceil((lon_num*drop_rate)) + 1))
    last = math.ceil((lon_num - span)/(span+1))
    last_ind = span + (last - 1)*(span + 1)
    
    drop_ind = np.linspace(span, last_ind, last)
    return list(set(range(lon_num)).difference(set(drop_ind)))

In [3]:
import iris
filename = iris.sample_data_path('uk_hires.pp')
cubes = iris.load(filename)
cube1 = cubes[0][0,0]

In [5]:
lat = iris.Constraint(grid_latitude = cube1.coord("grid_latitude").points[lat_id(cube1, 0.1)])

In [6]:
lon = iris.Constraint(grid_longitude = cube1.coord("grid_longitude").points[lon_id(cube1, 0.1)])

In [8]:
print(cube1.extract(lat & lon))

air_potential_temperature / (K)     (grid_latitude: 184; grid_longitude: 169)
     Dimension coordinates:
          grid_latitude                           x                    -
          grid_longitude                          -                    x
     Auxiliary coordinates:
          surface_altitude                        x                    x
     Derived coordinates:
          altitude                                x                    x
     Scalar coordinates:
          forecast_period: 6.0 hours
          forecast_reference_time: 2009-11-19 04:00:00
          level_height: 5.0 m, bound=(0.0, 13.3333) m
          model_level_number: 1
          sigma: 0.999424, bound=(1.0, 0.998464)
          time: 2009-11-19 10:00:00
     Attributes:
          STASH: m01s00i004
          source: Data from Met Office Unified Model
          um_version: 7.3


下のようなことをしなくてもcube.extract(Constraint)メソッドでできるわ

In [ ]:
# cubeは二次元のみ扱うことにする
def change_resolution(cube, drop_rate):
    lat_remain = lat_id(cube, drop_rate)
    lon_remain = lon_id(cube, drop_rate)
    
    # cubeに対してはsliceしかできない（np.ix_()で要素にアクセスすることはできない）ので、numpy arrayまで落として要素だけ取ってくる
    # 他のcoordinatesはそのまま新しいcubeに入れ込む
    extracted_data = cube.data[np.ix_(lat_remain, lon_remain)]
    
    # 新しいcubeようにlatとlonをクラスに入れる
    original_lat = cube.coord('grid_latitude').points
    original_lon = cube.coord('grid_longitude').points
    latitude = DimCoord(original_lat[lat_remain], standard_name='grid_latitude', units='degrees')
    longitude = DimCoord(original_lon[lon_remain], standard_name='grid_longitude', units='degrees')
    
    # lat, lon以外のcoordinateはそのまま新しいcubeに引き継ぐ
    # 上で作った新しいcubeに足していく
    # のはやめよう。cubeの次元を追加するのは難しいので一気に作る。
    dimcoords = [coord for coord in cube.coords() if coord.name() != "grid_latitude" and coord.name() != "grid_longitude" and str(type(coord)) == "<class 'iris.coords.DimCoord'>"]
    auxcoords = [coord for coord in cube.coords() if coord.name() != "grid_latitude" and coord.name() != "grid_longitude" and str(type(coord)) == "<class 'iris.coords.AuxCoord'>"]

    dims = [(i, n) for i, n in zip(dimcoords, range(2, len(dimcoords)))]
    lat_lon = [(latitude, 0),(longitude, 1)]
    dim_coords_list = lat_lon + dims
    aux_coords_list = [(i, n) for i, n in zip(auxcoords, range(len(auxcoords)))]
    
    # これだと次元を持つcoordinateの値がextracted_dataの中に含まれてないのでcube二できない
    # 入れるデータにそれぞれのcoordsが持つ値を入れてあげてdataをcubeに渡す。
    return Cube(extracted_data, dim_coords_and_dims=dim_coords_list, aux_coords_and_dims = aux_coords_list)